In [2]:
import pandas as pd
import numpy as np

from itertools import product

My initial idea here was to think of a way to simulate filling in the blocks. What I decided was I was not going to actually draw the grid or anything, but instead just count out the blocks. 

I also thought of a recursive solution but the sub-problems may not look like the original problem, and there would be multiple ways to split up a large block into small blocks, each giving different solutions.

The concept of my program is to treat each grid as a coordinate system, where each lattice point of the grid gets a coordinate. Suppose we have an $m \times n$ grid, so the top left corner is $(0,0)$ and the bottom right is $(m,n)$. We start at $(0,0)$ and track our current position with $(x,y)$. Then for a given block width and height, $(w,h)$, we can fill in the rectangle sequentially, incrementing $y$ by $1$ (i.e., going horizontally), and if there's an overhang (i.e., $y > n$), then increment $x$ by $1$ (i.e., move $1$ unit vertically) and set $y = 0$ (i.e., start over horiztonally). Finally, if there is overhand in both directions (i.e., $x > m$), then you would break and try a new $(w,h)$. This first algorithm is quite fast and can deal with large enough numbers for this problem. 

However, there is one speedup that only becomes obvious if you visualize the problem and do some math: I don't need the while loop in my program! In fact, based on the grid dimensions $(m,n)$ and the block dimensions $(w,h)$, I can just calculate the number of blocks that would be able to go horizontally, $n - w + 1$, and then multiply by the number of rows it would fit vertically, $m - h + 1$.

In [3]:
def get_rect_count(m,n):
    '''Given grid of size (m x n), count number of sub rectangles it has.'''
    blocks_checked = set()
    blocks = 0

    for height,width in product(range(m), range(n)):
            # i = block height, j = block width
            if (height, width) in blocks_checked:
                continue

            blocks_checked.add((height, width))
            blocks += (n - width)*(m - height)

            # OLD VERSION OF THE SKELETON OF THE SOLUTION
            # curr_x = x of top left corner
            # curr_y = y of top left corner
            #
            # e.g., a 2x2 grid would look like this 
            #
            #  (0,0) ---- (1,0) ---- (2,0)
            #    |          |          |
            #    |          |          |
            #  (0,1) ---- (1,1) ---- (2,1)
            #    |          |          |
            #    |          |          |
            #  (0,2) ---- (1,2) ---- (2,2)
            #
            # curr_x, curr_y = 0,0
            # while True:
            #     # if the block has gone all the way vertically, then break
            #     if not (0 <= curr_x + height <= m):
            #         break
                
            #     # the block has gone all the way horizontally, move down vertically and
            #     # start over horiztonally
            #     if not (0 <= curr_y + width <= n):
            #         blocks_checked[(height, width)] = blocks
            #         curr_x += 1
            #         curr_y = 0
            #         continue

            #     # print((curr_x, curr_y))

            #     blocks += 1
            #     curr_y += 1

    return blocks

* It's relatively quick to show that both a $2000 \times 1$ and a $1999 \times 1$ grid are just $1000$ away from $2 \times 10^6$. So that means the largest width we need to check is $2000$. 
* Also, because of the previous result, and because the area will always be part of the sum (since we have to count $1 \times 1$ rectangles), if the area is $>2 \times 10^6 + 1000$, then we don't need to check.
* And, finally, we only need to check when $n \geq m$, since if $m > n$, there is an identical rectangle where we just switch $n$ and $m$.
* With that previous requirement, we can also see that when $m = 53$, then any number $n \geq 53$ will violate the first condition. So we can stop at $m = 53$.

In [29]:
best_m = 2000
best_n = 1
best_blocks = get_rect_count(best_m, best_n)

for m in range(1,53):
    if m == 53:
        break

    for n in range(m, 2001 - m):
        if m*n > best_blocks:
            break

        blocks = get_rect_count(m, n)
        if abs(blocks - 2 * 10**6) < abs(best_blocks - 2 * 10**6):
            best_blocks = blocks
            best_m = m
            best_n = n
        
        if blocks > 2 * 10**6:
            break

print(best_blocks,m,n)

1999998 52 54
